In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import * 
import numpy as np

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
sc = spark.sparkContext

22/07/16 10:09:13 WARN Utils: Your hostname, Rutvijs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.119 instead (on interface en0)
22/07/16 10:09:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/16 10:09:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Data Fetched From Canada Open Government Data Website

Link: https://open.canada.ca/data/en/dataset/4301f4bb-1daa-4b50-afab-d1193b5d2284

In [2]:
# Load csv's

path = "./data/*.csv"

df = spark.read.format("csv")       \
    .option("header","true")        \
    .option("inferSchema","True")   \
    .load(path)

df.count()

8832277

In [3]:
df.printSchema()

root
 |-- LEVEL1ID: integer (nullable = true)
 |-- LEVEL2ID: integer (nullable = true)
 |-- LEVEL3ID: integer (nullable = true)
 |-- LEVEL4ID: integer (nullable = true)
 |-- LEVEL5ID: integer (nullable = true)
 |-- SURVEYR: integer (nullable = true)
 |-- BYCOND: string (nullable = true)
 |-- DESCRIP_E: string (nullable = true)
 |-- DESCRIP_F: string (nullable = true)
 |-- DEMCODE: string (nullable = true)
 |-- QUESTION: string (nullable = true)
 |-- TITLE_E: string (nullable = true)
 |-- TITLE_F: string (nullable = true)
 |-- ANSWER1: string (nullable = true)
 |-- ANSWER2: string (nullable = true)
 |-- ANSWER3: string (nullable = true)
 |-- ANSWER4: string (nullable = true)
 |-- ANSWER5: string (nullable = true)
 |-- ANSWER6: string (nullable = true)
 |-- ANSWER7: string (nullable = true)
 |-- MOST_POSITIVE_OR_LEAST_NEGATIVE: string (nullable = true)
 |-- NEUTRAL_OR_MIDDLE_CATEGORY: string (nullable = true)
 |-- MOST_NEGATIVE_OR_LEAST_POSITIVE: string (nullable = true)
 |-- AGREE: stri

In [4]:
dropp = [
    'DESCRIP_F',
    'TITLE_F',
    'DEPT_F',
    'INDICATORFRA',
    'SUBINDICATORFRA'
]

df = df.drop(*dropp)

In [5]:
def clean_title(x):
    y = str(x).split(".",maxsplit=1)
    return y[1]

clean_title_udf = udf(clean_title, StringType())

df = df.withColumn("TITLE_E", clean_title_udf("TITLE_E"))
df.select("TITLE_E").take(1)

[Row(TITLE_E=' I have the tools, technology and equipment I need to do my job.')]

In [6]:
df.select("QUESTION").take(2)

[Row(QUESTION='Q01'), Row(QUESTION='Q01')]

In [7]:
ls = [
"ANSWER1",
"ANSWER2",
"ANSWER3",
"ANSWER4",
"ANSWER5",
"ANSWER6",
"ANSWER7"
]

def remove_single_char_values(x):
    
    if x == " " or x is None:
        return int(-1)
    else:
        return int(x)

remove_single_char_values_udf = udf(remove_single_char_values, IntegerType() )

for x in ls:
    df = df.withColumn(x, remove_single_char_values_udf(x))


In [8]:

def remove_outlier_values(x):
    
    if x > 100:
        return int(0)
    else:
        return int(x)

remove_outlier_values_udf = udf(remove_outlier_values, IntegerType() )

for x in ls:
    df = df.withColumn(x, remove_outlier_values_udf(x))


In [9]:
def remove_Question_char_values(x):
    if x ==" " or x == "":
        return "ClearValue"
    else:
        return str(x)

remove_Question_char_values_udf = udf(remove_Question_char_values, StringType())

df = df.withColumn("QUESTION", remove_Question_char_values_udf("QUESTION"))

In [10]:
ls = [
"DEMCODE",
"BYCOND"
]

def remove_unknown_char_values(x):
    if x == " " or x == "":
        return "Unknown"
    else:
        return x

remove_unknown_char_values_udf = udf(lambda x: "Unknown" if x == " " or x == "" else x, StringType())

for x in ls:
    df = df.withColumn(x, remove_unknown_char_values_udf(x))

In [ ]:
ls = [
"MOST_POSITIVE_OR_LEAST_NEGATIVE",
"NEUTRAL_OR_MIDDLE_CATEGORY",
"MOST_NEGATIVE_OR_LEAST_POSITIVE",
"AGREE",
"ANSCOUNT",
"SCORE100",
]

def convert_to_int(x):
    return int(x)

convert_to_int_udf = udf(convert_to_int, IntegerType())

for x in ls:
    df = df.withColumn(x, convert_to_int_udf(x))

In [ ]:

def convert_to_float(x):
    return float(x)

convert_to_float_udf = udf(convert_to_float, IntegerType())

df = df.withColumn("SCORE5", convert_to_float_udf("SCORE5"))

In [66]:
def LEVEL_STRING(x1, x2, x3, x4, x5):
    return int(x1 + x2 + x3 + x4 + x5 )

levels = [
    "LEVEL1ID",
    "LEVEL2ID",
    "LEVEL3ID",
    "LEVEL4ID",
    "LEVEL5ID",
]

LEVEL_STRING_udf = udf(LEVEL_STRING, IntegerType())
convert_to_string_udf = udf(lambda x: str(x), StringType())
df = df.withColumn("LEVEL", 
                        LEVEL_STRING_udf(
                            convert_to_string_udf("LEVEL1ID"),
                            convert_to_string_udf("LEVEL2ID"),
                            convert_to_string_udf("LEVEL3ID"),
                            convert_to_string_udf("LEVEL4ID"),
                            convert_to_string_udf("LEVEL5ID"),
                        )
                    )

df.select(
    "LEVEL",
    "LEVEL1ID",
    "LEVEL2ID",
    "LEVEL3ID",
    "LEVEL4ID",
    "LEVEL5ID",
    ).show(25)

In [11]:
# df.printSchema()

In [69]:
df.count()


2666586

In [33]:
df = df.filter(df.ANSWER1 >= 0).filter(df.SCORE5 <= 5).filter(df.SCORE100 <= 100)
